<h1>Import Required Libraries</h1>

In [1]:
import yfinance as yf
import pickle
import numpy as np
import pandas as pd


<h1>Select the Stock</h1>

In [2]:
ticker = "INFY.NS"
print(f"\n Predicting signal for {ticker}")



 Predicting signal for INFY.NS


<h1>Load Saved Model Files</h1>

In [3]:
model = pickle.load(open(f"{ticker}_model.pkl", "rb"))
scaler = pickle.load(open(f"{ticker}_scaler.pkl", "rb"))
features = pickle.load(open(f"{ticker}_features.pkl", "rb"))


c:\Users\Janan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:440: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.5.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Janan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:440: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.5.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Janan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:440: InconsistentVersionWarning: Trying to 

<h1>Fetch Recent Market Data</h1>

In [4]:
data = yf.download(ticker, period="3mo")

if data.empty:
    raise ValueError("⚠ No recent data available")


[*********************100%***********************]  1 of 1 completed


<h1>Feature Engineering</h1>

In [5]:
data['Return'] = data['Close'].pct_change()
data['MA20'] = data['Close'].rolling(20).mean()
data['MA50'] = data['Close'].rolling(50).mean()
data['Volatility'] = data['Return'].rolling(20).std()
data['Volume_Change'] = data['Volume'].pct_change()

# RSI Calculation
delta = data['Close'].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)
avg_gain = gain.rolling(14).mean()
avg_loss = loss.rolling(14).mean()
rs = avg_gain / avg_loss
data['RSI'] = 100 - (100 / (1 + rs))


<h1>Clean the Data</h1>

In [6]:
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.dropna(inplace=True)

if len(data) == 0:
    raise ValueError("Not enough recent data after preprocessing.")


<h1>Prepare Latest Feature Row</h1>

In [7]:
X_latest = data[features].iloc[-1:]
print("\n Latest Feature Values:")
print(X_latest)



 Latest Feature Values:
Price         Return         MA20         MA50 Volatility Volume_Change  \
Ticker                                                                    
Date                                                                      
2026-02-24 -0.039171  1474.734991  1571.440002   0.021532      1.160047   

Price            RSI  
Ticker                
Date                  
2026-02-24  8.378656  


<h1>Scale the Features</h1>

In [8]:
X_scaled = scaler.transform(X_latest)


<h1>Predict Signal</h1>

In [9]:
signal = model.predict(X_scaled)[0]


<h1>Print Human-Readable Output</h1>

In [10]:
if signal == 1:
    print("\n📈 BUY Signal for INFY.NS")
elif signal == -1:
    print("\n📉 SELL Signal for INFY.NS")
else:
    print("\n⏸ HOLD Signal for INFY.NS")



📉 SELL Signal for INFY.NS


<h1>Save the Infosys Stock data in </h1>

In [11]:
data.to_csv("signal.csv")